In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import requests
from bs4 import BeautifulSoup

from scripts.preparation import download_data, add_macd, add_moving_average, add_psar

In [2]:
def plot_signal(stock_data, turn_idx, macd_up_idx, macd_down_idx):
    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.02,
        row_width=[0.4, 0.6],
    )

    candlestick = go.Candlestick(
        x=stock_data.index,
        open=stock_data["Open"],
        high=stock_data["High"],
        low=stock_data["Low"],
        close=stock_data["Close"],
        name="Candles",
    )

    fig.add_trace(
        candlestick,
        row=1,
        col=1,
    )

    for ema in ["ema5", "ema20", "ema60"]:
        fig.add_trace(
            go.Scatter(
                x=stock_data.index,
                y=stock_data[ema],
                mode="lines",
                name=ema,
            ),
            row=1,
            col=1,
        )

    for bar_plot in ["macd_1d_diff", "macd_2d_diff"]:
        fig.add_trace(
            go.Bar(
                x=stock_data.index,
                y=stock_data[bar_plot],
                name=bar_plot,
            ),
            row=2,
            col=1,
        )

    for line_plot in ["macd_ma3", "macd_ma5"]:
        fig.add_trace(
            go.Scatter(
                x=stock_data.index,
                y=stock_data[line_plot],
                mode="lines",
                name=line_plot,
            ),
            row=2,
            col=1,
        )

    scatter_minima = go.Scatter(
                x=stock_data.index[turn_idx],
                y=stock_data["ema5"][turn_idx],
                mode="markers",
                marker=dict(color="red", size=8),
                name="turn",
            )
    
    macd_up = go.Scatter(
                x=stock_data.index[macd_up_idx],
                y=stock_data["ema5"][macd_up_idx] * 0.95,
                mode="markers",
                marker=dict(color="black", size=8),
                name="macd_up",
            )
    
    macd_down = go.Scatter(
                x=stock_data.index[macd_down_idx],
                y=stock_data["High"][macd_down_idx] * 1.05,
                mode="markers",
                marker=dict(color="blue", size=8),
                name="macd_down",
            )

    fig.add_trace(
                scatter_minima,
                row=1,
                col=1,
            )
    
    fig.add_trace(
                macd_up,
                row=1,
                col=1,
            )
    
    fig.add_trace(
                macd_down,
                row=1,
                col=1,
            )

    fig.update_layout(
        autosize=True,
        xaxis_rangeslider_visible=False,
        height=800,
    )

    fig.show()


# screener testing

In [3]:
def extract_ticker(df_row, russell_list):
    capital = 0
    for letter in df_row:
        if letter.isupper():
            capital += 1
        else:
            break
    
    answer = df_row[:capital-1]
    white_list = ["SMCI", "MSTR"]

    while len(answer) > 0:
        if answer in (russell_list + white_list):
            return answer
        else:
            answer = answer[:len(answer)-1]
    
    print("Can't find: ", answer)
    return None


russell_table = pd.read_html("https://en.wikipedia.org/wiki/Russell_1000_Index")
russell_list = list(russell_table[2]["Ticker"])


top100 = pd.read_html("https://www.tradingview.com/markets/stocks-usa/market-movers-active/")
ticker_list = top100[0]["Symbol"].apply(extract_ticker, russell_list=russell_list)

ticker_object = download_data(ticker_list)

In [4]:
# ticker_list = ["SMCI", "MARA", "SOFI"]

# ticker_object = download_data(ticker_list)

In [10]:
count = 1
for ticker in ticker_list:
    stock_data = ticker_object.tickers[ticker].history(period="200d")
    stock_data = add_macd(stock_data)
    stock_data = add_moving_average(stock_data)
    stock_data = add_psar(stock_data)
    stock_data["ema_bull"] = True
    for i in ["ema5", "ema20", "ema60"]:
        stock_data[f"{i}_bull"] = stock_data[i] > stock_data[i].shift(1)
        stock_data["ema_bull"] *= stock_data[f"{i}_bull"]

    stock_data["macd_1d_up"] = (stock_data["macd_1d_diff"] > stock_data["macd_1d_diff"].shift(1)).rolling(2).sum()
    stock_data["macd_2d_up"] = (stock_data["macd_2d_diff"] > stock_data["macd_2d_diff"].shift(1)).rolling(2).sum()
    macd_up_idx = (stock_data["macd_1d_up"] >= 2) * (stock_data["macd_2d_up"] >= 2)

    stock_data["macd_1d_down"] = (stock_data["macd_1d_diff"] < stock_data["macd_1d_diff"].shift(1)).rolling(2).sum()
    stock_data["macd_2d_down"] = (stock_data["macd_2d_diff"] < stock_data["macd_2d_diff"].shift(1)).rolling(2).sum()
    macd_down_idx = (stock_data["macd_1d_down"] >= 2) * (stock_data["macd_2d_down"] >= 2)

    stock_data["macd_1d_turn"] = np.where(stock_data["macd_1d_diff"] < 0, 0, 1)
    stock_data["macd_2d_turn"] = np.where(stock_data["macd_2d_diff"] < 0, 0, 1)
    stock_data["macd_turn_signal"] = stock_data["macd_1d_turn"] * stock_data["macd_2d_turn"]
    stock_data["psar_signal"] = (stock_data["psar"] / stock_data["Close"]) < 1

    stock_data["final_signal"] = stock_data["macd_turn_signal"] * stock_data["psar_signal"]

    soft_rules = stock_data["ema_bull"] + stock_data["psar_signal"]

    turn_idx = (stock_data["macd_turn_signal"] > 0) * soft_rules

    if (sum(turn_idx.tail(2)) > 0 and sum(macd_down_idx.tail(3)) == 0 and sum(turn_idx.tail(5)) < 4):
        print(ticker)
        plot_signal(stock_data, turn_idx, macd_up_idx, macd_down_idx)
        count += 1

    # if count > 10:
    #     break
print(count)


V


XOM


JPM


MRK


UNH


...

COP


HUM


CSCO


EMR


25
